In [1]:
import torch

index_cls = [i for i in range(15)]
index_cls.append(20)
index_reg = [i for i in range(4 * 15)]

checkpoint_novel = torch.load('work_dirs/voc/tfa/split3/tfa_r101_fpn_voc-split3_base-training_20211031_114840_random-init-bbox-head-9bb8c09b.pth')
checkpoint_base = checkpoint_novel.copy()

checkpoint_base['state_dict']['roi_head.bbox_head.fc_cls.weight'] = checkpoint_novel['state_dict']['roi_head.bbox_head.fc_cls.weight'][index_cls]
checkpoint_base['state_dict']['roi_head.bbox_head.fc_cls.bias'] = checkpoint_novel['state_dict']['roi_head.bbox_head.fc_cls.bias'][index_cls]
checkpoint_base['state_dict']['roi_head.bbox_head.fc_reg.weight'] = checkpoint_novel['state_dict']['roi_head.bbox_head.fc_reg.weight'][index_reg]
checkpoint_base['state_dict']['roi_head.bbox_head.fc_reg.bias'] = checkpoint_novel['state_dict']['roi_head.bbox_head.fc_reg.bias'][index_reg]
torch.save(checkpoint_base, 'work_dirs/voc/tfa/split3/tfa_r101_fpn_voc-split3_base-training_20211031_114840_base-class-bbox-head-9bb8c09b.pth')

In [1]:
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '4'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

#!PORT=29500 ./tools/detection/dist_train.sh configs/detection/tfa/voc/split1/tfa_r101_fpn_voc-split1_3shot-fine-tuning-my.py 2
%run tools/detection/train.py configs/detection/tfa/voc/split3/tfa_r101_fpn_voc-split3_base-training-my.py

fatal: Not a git repository (or any parent up to mount point /data)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
2022-03-05 10:39:00,401 - mmfewshot - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.11 (default, Jul 27 2021, 14:32:16) [GCC 7.5.0]
CUDA available: True
GPU 0: GeForce RTX 2080 Ti
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 10.0, V10.0.130
GCC: gcc (Ubuntu 5.4.0-6ubuntu1~16.04.12) 5.4.0 20160609
PyTorch: 1.7.0
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) oneAPI Math Kernel Library Version 2021.4-Product Build 20210904 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.6.0 (Git Hash 5ef631a030a6f73131c77892041042805a06064f)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 10.1
  - NVCC architecture flags: -gencode;arch=compute_37,co

base statistics in base_statistics/2022-03-05/10:38:58


2022-03-05 10:39:38,537 - mmfewshot - INFO - Epoch [1][50/7159]	lr: 0.000e+00, eta: 0:49:34, time: 0.418, data_time: 0.052, memory: 6089, loss_rpn_cls: 0.0063, loss_rpn_bbox: 0.0149, loss_cls: 0.0719, acc: 97.0527, loss_bbox: 0.1298, loss: 0.2229
2022-03-05 10:39:56,995 - mmfewshot - INFO - Epoch [1][100/7159]	lr: 0.000e+00, eta: 0:46:21, time: 0.370, data_time: 0.009, memory: 6089, loss_rpn_cls: 0.0040, loss_rpn_bbox: 0.0108, loss_cls: 0.0689, acc: 97.2539, loss_bbox: 0.1232, loss: 0.2069
2022-03-05 10:40:15,290 - mmfewshot - INFO - Epoch [1][150/7159]	lr: 0.000e+00, eta: 0:44:56, time: 0.366, data_time: 0.008, memory: 7100, loss_rpn_cls: 0.0043, loss_rpn_bbox: 0.0137, loss_cls: 0.0723, acc: 97.0723, loss_bbox: 0.1274, loss: 0.2176
2022-03-05 10:40:33,205 - mmfewshot - INFO - Epoch [1][200/7159]	lr: 0.000e+00, eta: 0:43:52, time: 0.359, data_time: 0.008, memory: 7281, loss_rpn_cls: 0.0069, loss_rpn_bbox: 0.0140, loss_cls: 0.0714, acc: 97.1699, loss_bbox: 0.1273, loss: 0.2196
2022-03-0

In [2]:
import os
import sys
import pickle
import random
import numpy as np

#need = 10000
base_class_path = '/data/zhaozhiyuan/mmfewshot-main/base_statistics/2022-03-05/10:38:58'
classes = os.listdir(base_class_path)
print(classes)
base_statistics = {}

for i,c in enumerate(classes):
    print('{}/{}'.format(i,c))
    if '.' in c:
        continue
    feats = os.listdir(os.path.join(base_class_path, c))
    #need = min(10000, len(feats))
    #feats = random.sample(feats, need)
    
    embeds = []
    # all features
    for feat in feats:
        f = open(os.path.join(base_class_path, c, feat), 'rb')
        embed = pickle.load(f)
        embeds.append(embed)
        f.close()
    
    mean = np.mean(embeds, axis=0)
    cov = np.cov(np.array(embeds).T)
    base_statistics[c] = {}
    base_statistics[c]['mean'] = mean
    base_statistics[c]['variance'] = cov
    base_statistics[c]['N'] = len(embeds)
    
    """
    # mean & variance
    dim = 1024
    mean = np.zeros(embeds[0].shape)
    for embed in embeds:
        mean += embed
    mean /= len(embeds)
    
    variance = np.zeros((dim,dim))
    for embed in embeds:
        minus = np.expand_dims((embed - mean), axis = 1)
        v = np.matmul(minus, minus.T)
        variance += v
    variance /= (len(embeds) - 1)
    base_statistics[c] = {}
    base_statistics[c]['mean'] = mean
    base_statistics[c]['variance'] = variance
    base_statistics[c]['N'] = len(embeds)
    """
    
    """
    归一化
    """
    """
    # mean
    dim = 1024
    mean = np.zeros(embeds[0].shape)
    for embed in embeds:
        mean += embed
    mean /= len(embeds)
    # 每一维的标准差
    embeds_norm = [np.expand_dims((embed - mean), axis = 0) for embed in embeds]
    embeds_norm = np.concatenate(embeds_norm, axis=0)
    std = np.std(embeds_norm, axis=0)
    
    #embeds_norm = [embed/std for embed in embeds_norm]
    
    dim = 1024
    mean = np.zeros(embeds[0].shape)
    for embed in embeds_norm:
        mean += embed
    mean /= len(embeds_norm)
    
    variance = np.zeros((dim,dim))
    for embed in embeds_norm:
        minus = np.expand_dims(embed, axis = 1)
        v = np.matmul(minus, minus.T)
        variance += v
    variance /= (len(embeds_norm) - 1)
    base_statistics[c] = {}
    base_statistics[c]['mean'] = mean
    base_statistics[c]['variance'] = variance
    base_statistics[c]['N'] = len(embeds)
    """ 
    
f = open(os.path.join(base_class_path, 'base_statistics.pt'), 'wb')
pickle.dump(base_statistics, f)
f.close()

['12', '14', '8', '6', '3', '11', '4', '5', '2', '13', '0', '9', '10', '1', '7']
0/12
1/14
2/8
3/6
4/3
5/11
6/4
7/5
8/2
9/13
10/0
11/9
12/10
13/1
14/7


In [ ]:
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '4'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

#!PORT=29500 ./tools/detection/dist_train.sh configs/detection/tfa/voc/split1/tfa_r101_fpn_voc-split1_3shot-fine-tuning-my.py 2
%run tools/detection/train.py configs/detection/tfa/voc/split3/10shot/tfa_r101_fpn_voc-split3_10shot-fine-tuning-my-test-ema.py

fatal: Not a git repository (or any parent up to mount point /data)
2022-03-06 05:42:14,887 - mmfewshot - INFO - Environment info:
------------------------------------------------------------
sys.platform: linux
Python: 3.7.11 (default, Jul 27 2021, 14:32:16) [GCC 7.5.0]
CUDA available: True
GPU 0: GeForce RTX 2080 Ti
CUDA_HOME: /usr/local/cuda
NVCC: Cuda compilation tools, release 10.0, V10.0.130
GCC: gcc (Ubuntu 5.4.0-6ubuntu1~16.04.12) 5.4.0 20160609
PyTorch: 1.7.0
PyTorch compiling details: PyTorch built with:
  - GCC 7.3
  - C++ Version: 201402
  - Intel(R) oneAPI Math Kernel Library Version 2021.4-Product Build 20210904 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v1.6.0 (Git Hash 5ef631a030a6f73131c77892041042805a06064f)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 10.1
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_50;-gencode;arch=compute_60,code=

k=15 alpha=0.0001 repeat=1
beta_novel=0.999999 beta_base=0.999999


2022-03-06 05:42:26,861 - mmfewshot - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'open-mmlab://detectron2/resnet101_caffe'}
2022-03-06 05:42:26,866 - mmcv - INFO - load model from: open-mmlab://detectron2/resnet101_caffe
2022-03-06 05:42:26,867 - mmcv - INFO - load checkpoint from openmmlab path: open-mmlab://detectron2/resnet101_caffe
2022-03-06 05:42:27,440 - mmfewshot - INFO - initialize FPN with init_cfg [{'type': 'Caffe2Xavier', 'override': {'type': 'Caffe2Xavier', 'name': 'lateral_convs'}}, {'type': 'Caffe2Xavier', 'override': {'type': 'Caffe2Xavier', 'name': 'fpn_convs'}}]
2022-03-06 05:42:27,477 - mmfewshot - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2022-03-06 05:42:27,488 - mmfewshot - INFO - initialize CosineSimBBoxHeadMyTuneTestEMA with init_cfg [{'type': 'Caffe2Xavier', 'override': {'type': 'Caffe2Xavier', 'name': 'shared_fcs'}}, {'type': 'Normal', 'override': {'type': 'Normal', 'name': 'fc_c

[>>>                         ] 540/4952, 11.1 task/s, elapsed: 49s, ETA:   399s